Кирилл Лалаянц, R33352

# Лабораторная работа No5
## Типовые динамические звенья

Вариант 6

Импорт необходимых для работы библиотек. 

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import control 
import sympy
import os
from tbcontrol.symbolic import routh
import math
import tqdm

SAVE_PATH = 'tex-report/figs/'
os.makedirs(SAVE_PATH, exist_ok=True)

sympy.init_printing()
p = sympy.Symbol("p")
s = sympy.Symbol("s")
t = sympy.Symbol("t")
w = sympy.Symbol("w")
I = sympy.I


def get_t(end_t = 10, dt=0.001, start_t = 0):
    return np.linspace(start_t, end_t, int(end_t / dt))

In [ ]:
def plot_bode(ss, theory_lambda, freqs, plot_name, save_name, max_t = 10, fsize=40):
    mag, phase, omega = control.bode(ss, freqs, plot=False)
    freqs = np.array(freqs)
    theory = theory_lambda(freqs)
    mag_theory = np.absolute(theory)
    phase_theory = np.angle(theory)
    ts = [i/1000 for i in range(0, 1000 * max_t)]
    
    
    fig, axs = plt.subplots(2, 3, figsize=(20, 20))
    fig.suptitle(plot_name, fontsize=fsize , y=0.95)
    axs[0, 0].plot(omega, mag, linewidth=12, label='experiment')
    axs[0, 0].plot(omega, mag_theory, 'r--', linewidth=4, label='theory')
    axs[0, 0].set_title('AЧХ', fontsize=fsize)
    axs[0, 0].grid(True)
    axs[0, 0].set_xlabel(f"Частота, [рад/c]", fontsize=fsize // 2)
    axs[0, 0].set_ylabel(f"Амплитуда", fontsize=fsize // 2)
    axs[0, 0].tick_params(axis='both', which='major', labelsize=fsize // 2)
    axs[0, 0].tick_params(axis='both', which='minor', labelsize=fsize // 2)
    axs[0, 0].legend(fontsize=20)
    
    
    axs[0, 1].plot(omega, phase, linewidth=12, label='experiment')
    axs[0, 1].plot(omega, phase_theory, 'r--', linewidth=4, label='theory')
    axs[0, 1].set_title('ФЧХ', fontsize=fsize)
    axs[0, 1].grid(True)
    axs[0, 1].set_xlabel(f"Частота, [рад/c]", fontsize=fsize // 2)
    axs[0, 1].set_ylabel(f"Фаза, [рад]", fontsize=fsize // 2)
    axs[0, 1].tick_params(axis='both', which='major', labelsize=fsize // 2)
    axs[0, 1].tick_params(axis='both', which='minor', labelsize=fsize // 2)
    axs[0, 1].legend(fontsize=20)    
    
    axs[1, 0].plot(omega, mag, linewidth=12, label='experiment')
    axs[1, 0].plot(omega, mag_theory, 'r--', linewidth=4, label='theory')
    axs[1, 0].set_title('ЛAЧХ', fontsize=fsize)
    axs[1, 0].set_xscale('log')
    axs[1, 0].set_yscale('log')
    axs[1, 0].grid(True, which="both", ls="-", color='0.65')
    axs[1, 0].set_xlabel(f"Частота, [рад/c]", fontsize=fsize // 2)
    axs[1, 0].set_ylabel(f"Амплитуда, [дБ]", fontsize=fsize // 2)
    axs[1, 0].tick_params(axis='both', which='major', labelsize=fsize // 2)
    axs[1, 0].tick_params(axis='both', which='minor', labelsize=fsize // 2)  
    axs[1, 0].legend(fontsize=20)  
    
    axs[1, 1].plot(omega, phase, linewidth=12, label='experiment')
    axs[1, 1].plot(omega, phase_theory, 'r--', linewidth=4, label='theory')
    axs[1, 1].set_title('ЛФЧХ', fontsize=fsize)
    axs[1, 1].set_xscale('log')
    axs[1, 1].grid(True, which="both", ls="-", color='0.65')
    axs[1, 1].set_xlabel(f"Частота, [рад/c]", fontsize=fsize // 2)
    axs[1, 1].set_ylabel(f"Фаза, [рад]", fontsize=fsize // 2)
    axs[1, 1].tick_params(axis='both', which='major', labelsize=fsize // 2)
    axs[1, 1].tick_params(axis='both', which='minor', labelsize=fsize // 2)
    axs[1, 1].legend(fontsize=20)
    
    
    step_response = control.step_response(ss, ts).outputs
    axs[1, 2].plot(ts, step_response, linewidth=12, label='experiment')
    axs[1, 2].set_title('Step response', fontsize=fsize)
    axs[1, 2].set_xlabel(f"t, [c]", fontsize=fsize // 2)
    axs[1, 2].set_ylabel(f"y", fontsize=fsize // 2)
    axs[1, 2].tick_params(axis='both', which='major', labelsize=fsize // 2)
    axs[1, 2].tick_params(axis='both', which='minor', labelsize=fsize // 2)
    axs[1, 2].legend(fontsize=20)
    axs[1, 2].grid()
    
    
    impulse_response = control.impulse_response(ss, ts).outputs
    axs[0, 2].plot(ts, impulse_response, linewidth=12, label='experiment')
    axs[0, 2].set_title('Impulse response', fontsize=fsize)
    axs[0, 2].set_xlabel(f"t, [c]", fontsize=fsize // 2)
    axs[0, 2].set_ylabel(f"y", fontsize=fsize // 2)
    axs[0, 2].tick_params(axis='both', which='major', labelsize=fsize // 2)
    axs[0, 2].tick_params(axis='both', which='minor', labelsize=fsize // 2)
    axs[0, 2].legend(fontsize=20)
    axs[0, 2].grid()
    
    
    plt.savefig(f'{SAVE_PATH}/{save_name}.jpg') 
    
def get_ss_and_theory(subs, sA, sB, sC, sD):
    stf = (sC @ (sympy.diag([p] * sA.shape[0], unpack=True) - sA).inv() @ sB + sD)[0]
    wjw = stf.subs({p: I*w}).subs(subs)
    theory = sympy.lambdify(w, wjw, 'numpy')
    
    sympy_to_numpy = lambda mat, subs: np.array(mat.subs(subs)).astype(np.float64)
    A = sympy_to_numpy(sA, subs)
    B = sympy_to_numpy(sB, subs)
    C = sympy_to_numpy(sC, subs)
    D = sympy_to_numpy(sD, subs)
    
    return control.ss(A, B, C, D), theory


## Task 1

### 2. Brushed DC motor 2.0.

In [ ]:
t2_K_M, t2_K_E, t2_J, t2_R, t2_L = 0.3637, 0.3637, 0.0023, 4.6050, 1.1784
k_m, k_e, J, R, L = sympy.symbols('k_m k_e J R L')
t2_subs = {k_m: t2_K_M, k_e: t2_K_E, J: t2_J, R: t2_R, L: t2_L}

t2_sA = sympy.Matrix([[0, 1 / J], [-k_m * k_e / L, -R / L]])
t2_sB = sympy.Matrix([0, k_m / L])
t2_sC = sympy.Matrix([[1, 0]])
t2_sD = sympy.Matrix([0])

t2_ss, t2_theory = get_ss_and_theory(t2_subs, t2_sA, t2_sB, t2_sC, t2_sD)

print(control.ss2tf(t2_ss))

plot_bode(t2_ss, t2_theory, [i/10 for i in range(1000)], '2. Brushed DC motor 2.0.', '1_2')

### 3. Конденсируй. Интегрируй. Умножай.

In [ ]:
t3_C = 287 * 10 ** -6
C = sympy.symbols('C')
t3_subs = {C: t3_C}

t3_sA = sympy.Matrix([0])
t3_sB = sympy.Matrix([1 / C])
t3_sC = sympy.Matrix([1])
t3_sD = sympy.Matrix([0])

t3_ss, t3_theory = get_ss_and_theory(t3_subs, t3_sA, t3_sB, t3_sC, t3_sD)

print(control.ss2tf(t3_ss))

plot_bode(t3_ss, t3_theory, [i/10 for i in range(10, 1000)], '3. Конденсируй. Интегрируй. Умножай.', '1_3')

## Task 2

### 1. Brushed DC motor.

In [ ]:
t1_K_M, t1_K_E, t1_J, t1_R, t1_L = 0.3612, 0.3612, 0.0031, 4.7237, 1.0567
k_m, k_e, J, R = sympy.symbols('k_m k_e J R')
t1_subs = {k_m: t1_K_M, k_e: t1_K_E, J: t1_J, R: t1_R}

t1_sA = sympy.Matrix([-k_m * k_e / (J * R)])
t1_sB = sympy.Matrix([k_m / (J * R)])
t1_sC = sympy.Matrix([[1]])
t1_sD = sympy.Matrix([0])

t1_ss, t1_theory = get_ss_and_theory(t1_subs, t1_sA, t1_sB, t1_sC, t1_sD)

print(control.ss2tf(t1_ss))

plot_bode(t1_ss, t1_theory, [i/10 for i in range(1000)], '1. Brushed DC motor.', '2_1')

### 4. Tachogenerator.

In [ ]:
t4_R, t4_R_l, t4_L, t4_k_e = 5.2188, 263, 0.4577, 0.3427
R, R_l, L, k_e = sympy.symbols('R R_l L k_e')
t4_subs = {R: t4_R, R_l: t4_R_l, L: t4_L, k_e: t4_k_e}

t4_sA = sympy.Matrix([-(R_l + R) / L])
t4_sB = sympy.Matrix([k_e])
t4_sC = sympy.Matrix([1])
t4_sD = sympy.Matrix([0])

t4_ss, t4_theory = get_ss_and_theory(t4_subs, t4_sA, t4_sB, t4_sC, t4_sD)

print(control.ss2tf(t4_ss))

plot_bode(t4_ss, t4_theory, [i/10 for i in range(0, 1000)], '4. Tachogenerator.', '1_4')

### 5. Spring-mass system.

In [ ]:
t5_M, t5_k = 35, 324
m, k = sympy.symbols('m, k')
t4_subs = {m: t5_M, k: t5_k}

t4_sA = sympy.Matrix([[0, 1], [-k / m, 0]])
t4_sB = sympy.Matrix([[0], [1 / m]])
t4_sC = sympy.Matrix([[1, 0]])
t4_sD = sympy.Matrix([0])

t4_ss, t4_theory = get_ss_and_theory(t4_subs, t4_sA, t4_sB, t4_sC, t4_sD)

print(control.ss2tf(t4_ss))

plot_bode(t4_ss, t4_theory, [i/10 for i in range(0, 1000)], '5. Spring-mass system.', '1_5')